**Scraping**  
scraping : scale 결정 가능(내가원하는만큼), 특정사이트에서 내가 원하는부분만 긁어오는 것   
crawling : large scale (빅데이터), 네트워크를 끊임없이 돌아다니는 애

In [37]:
#url 받아오고 에러처리

import requests
import time
def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    
    try:
        resp = requests.request(method, url, params=param, data=data, headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"})
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries>0:  #500대 에러일시 서버잘못이니까 재시도 메세지 띄워줌
            print(maxretries)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretries-1)
        print(e.response.status_code)
        print(e.response.reason)
    return resp
    


In [38]:
#http://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu
from bs4 import BeautifulSoup

url = "http://www.ppomppu.co.kr/zboard/zboard.php"
param = {
    'id':'ppomppu',
    'page': 1
}
html = download('get',url,param)
dom = BeautifulSoup(html.text,'lxml')


#썸네일, title, 추천, 조회 가져오기
for tr in dom.select("tr[class^=list]")[1:]:   #tr인데 class가 list로 시작하는
    td = tr.find_all("td", recursive=False)
    print(requests.compat.urljoin(url, td[3].a.img["src"])) #썸네일이미지
    
    print(td[3].select("td[valign=middle] > a")[0].text) #타이틀
    temp = td[5].text.split("-")
    print((0,0) if len(temp) < 2 else ",".join(temp))  #추천, (추천 비추천 차이 많이나는것만 가져옴)
    print(td[6].text.strip()) #strip : 양옆에 공백 제거  #조회
    print()


http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/9/small_323889.jpg?t=20190716161444
[카카오 선물하기] 설레임130ml*24개(12900/무료)
(0, 0)
263

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/8/small_323888.jpg?t=20190716161143
[G마켓] 쁘띠첼 미초 900mlx4개 (13,300원/무료배송)
(0, 0)
152

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/7/small_323887.jpg?t=20190716154110
[G마켓] 스파게티소스600gx2 +커리/쌀국수소스x1+면500g (9,900원 / 무배)
(0, 0)
972

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/5/small_323885.jpg?t=20190716153432
[G9] 삼성공식인증점 갤럭시탭S5e 10.5 WiFi 128GB (488,700원/무료배송) + 상품평 작성시 해피머니1만원
0 , 3
2439

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/2/small_323882.jpg?t=20190716152450
[G마켓] LG 휘센 제습기 DQ168PBE (368,470원/ 무료배송)
(0, 0)
1175

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/0/small_323880.jpg?t=20190716151334
[셀티바] 다이어트유산균(48%할인/무료배송)
0 , 2
742

http://cdn.ppomppu.co.kr/zboard/data/_thumb/ppomppu/9/small_323879.jpg?t=20190716145420
[위메프] EVA 30CM 마사지 폼롤러 (5900/무료)
4 , 1
4576

http:/

**뽐뿌 자유게시판 10개페이지만 포커스 크롤링**


[Crawling]
1. Focused Crawling(10개페이지 내에서)
2. 각 페이지 당 게시글(제목) 부분의 a 수집
3. 수집된 a로부터 Scraping

[Scraping]
1. 제목, 본문내용, 댓글 파싱
(댓글은 있거나 여러개 있음)


In [42]:
#http://www.ppomppu.co.kr/zboard/zboard.php?id=freeboard
from bs4 import BeautifulSoup

url = "http://www.ppomppu.co.kr/zboard/zboard.php"
param = {
    'id': 'freeboard',
    'page': 1
}

def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    return [requests.compat.urljoin(url, _["href"]) for _ in dom.select("td.list_vspace > a") if _.has_attr('href') and len(_["href"])>3]

NameError: name 'seed' is not defined

In [40]:
[requests.compat.urljoin(url, _["href"]) for _ in dom.select("td.list_vspace > a") if _.has_attr('href') and len(_["href"])>3] 


['http://www.ppomppu.co.kr/zboard/view.php?id=regulation&page=1&divpage=202&no=6']

In [43]:
queue = []
queue.extend([requests.compat.urljoin(url, _["href"]) for _ in dom.select("td.list_vspace > a") if _.has_attr('href') and len(_["href"])>3]) 

seen = []
#queue에 어팬드 시켜서 큐에 더이상 데이터가 없을때까지 돌거임
while queue: #여긴항상트루
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL) #여기서 중복된 유알엘들이 들어옴. 제거해야함
    for link in linkList:
        if link not in queue and link not in seen:  #링크가 queue에도 없어야하고, seen에도 없어야해
            queue.append(link)
    
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))  
    #baseURL.expand(linkList) #붙임


Queue: 0, Seen: 1


In [30]:
seen

['http://www.ppomppu.co.kr/zboard/view.php?id=regulation&page=1&divpage=202&no=6']